In [ ]:
%%javascript
// Disables truncation of output window
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Config
output_dir = "trials1"
trial_type = "DEFAULT"
analysis_type = "temporal"
num_trials = 100
options = "-output_dir %s" % output_dir
options += " -type %s" % trial_type
options += " -analysis_type %s" % analysis_type
options += " -num_trials %d" % num_trials
options += " -summarize_only 1"  # enable to collate results of completed trials (sets of multiple simulations)
# options += " -analyze_results_only 1"  # enable to collate results of completed simulations

In [ ]:
from anamod.simulation import run_trials
outputs = run_trials.main(options)

In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from anamod.constants import FDR, POWER, TEMPORAL_FDR, TEMPORAL_POWER, AVERAGE_WINDOW_FDR, AVERAGE_WINDOW_POWER
from anamod.constants import TEMPORAL, WINDOW_OVERLAP

GROUPS = {"Overall Feature Importance Detection": (FDR, POWER),
          "Temporal Feature Importance Detection": (TEMPORAL_FDR, TEMPORAL_POWER),
          "Average Window Detection": (AVERAGE_WINDOW_FDR, AVERAGE_WINDOW_POWER)}


def visualize(data):
    """Visualize outputs"""
    if analysis_type == TEMPORAL:
        # Window overlap histogram
        fig = go.Figure()
        for param, values in data[WINDOW_OVERLAP].items():
            fig.add_trace(go.Histogram(x=values, name=param))
        fig.update_traces(histnorm="probability", xbins=dict(start=0.0, end=1.0), opacity=0.6)
        fig.update_layout(title={"text": "Histogram of Average Window Overlap", "xanchor": "center", "x": 0.5},
                          xaxis_title="Average Window Overlap", yaxis_title="Probability", template="none")
        fig.show()
    for name, group in GROUPS.items():
        fig = go.Figure()
        for cat in group:
            x, y = ([], [])
            for param, values in data[cat].items():
                y.extend(values)
                x.extend([param] * len(values))
            fig.add_trace(go.Violin(x=x, y=y,
                                    legendgroup=cat, scalegroup=cat, name=cat))
        fig.update_traces(box_visible=True, meanline_visible=True, opacity=0.6, points="all")
        fig.update_layout(title={"text": name, "xanchor": "center", "x": 0.5},
                          xaxis_title=trial_type, yaxis_title="Value",
                          violinmode="group", template="none")
        fig.show()

visualize(outputs)